# Homework 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from numpy.linalg import eig
from scipy.linalg import expm

## Question 1.a): Entanglement Entropy from Free Fermion Correlation Matrix

#### a) Building $N\times N$ Hamiltonian matrix $A$ 
Particle perserving quadratic fermionic Hamiltonian.

In [2]:
def build_matrix_A(N, bc=1): # bc=1 (default) for PBC and bc=-1 for APBC
    A = np.zeros((N, N)) 
    for n in range(N-1):
        A[n, n+1] = -1
        A[n+1, n] = -1
    A[N-1,0] = -bc
    A[0,N-1] = -bc
    return A

#### b) Building translation operator $T_f$### Periodic Boundary Conditions

In [3]:
def build_T_operator(N, bc=1): # bc=1 (default) for PBC and bc=-1 for APBC
    T_op = np.zeros((N,N))
    for n in range(N-1):
        T_op[n,n+1] = 1
        
    T_op[N-1,0] = bc
    return T_op

### Diagonalisation of  $A$ and $T$.

In [4]:
def obtain_M(A, T_op):
    N = int(np.shape(A)[0])
    H = A.dot(np.eye(N) + 0.13*T_op)+0.05*T_op
    D, U = eig(H)
    
    e = np.real(np.diag((U.conj().T).dot(A).dot(U)))
    k = np.angle(np.diag((U.conj().T).dot(T_op).dot(U)))
    
    ord_idx = np.argsort(e)
    e = e[ord_idx]
    k = k[ord_idx]
    U = U[:, ord_idx]
    
    M = (U.dot(np.diag(np.sign(e)))).dot(U.conj().T)
    
    return M

In [ ]:
def obtain_M_2(N, ):
    np.eye(N)-2/N

In [5]:
N = 100

A_apbc = build_matrix_A(N, bc = -1)
T_apbc = build_T_operator(N, bc = -1)

M = obtain_M(A_apbc, T_apbc)

print("Is M successfully real?", np.allclose(M, np.real(M)))
M = np.real(M) #cleaning neglible imaginary part
print("Is M successfully symmetric?", np.allclose(M, (M+M.conj().T)/2))

Is M successfully real? True
Is M successfully symmetric? True


In [6]:
np.shape(M)

(100, 100)

In [ ]:
def ent_entropy_fermi(M):
    p = np.shape(M)[0]
    entropy = np.zeros(p)
    for n in range(p+1):
        M_red = M[:n, :n]
        u,v = eig(M_red)
        for m in range(n):
            x = u[m]
            if abs(x) < 1:
                entropy[n-1] -= (1+x)/2*np.log2((1+x)/2) + (1-x)/2*np.log2((1-x)/2)
            elif abs(x) > 1+1e-5:
                print("warning: (absolute) single particle eigenvalue > 1")
    return entropy

In [ ]:
entropy_numerical = ent_entropy_fermi(M_A)

In [ ]:
cft_ent_entropy = [1/3*(np.log2(N/np.pi*np.sin(n*np.pi/N)))+1.05 for n in L_s] #c = 1
plt.title(r"Entanglement entropy $S(\rho)$", fontsize = 20)
plt.xscale('log')
plt.grid()
plt.ylabel(r"Entropy, $S(\rho)$", fontsize = 15)
plt.xlabel(r"Parittion, $L$", fontsize = 15)
plt.plot(L_s, entropy_numerical, '.', ms=7, label = 'numerical')
plt.plot(L_s, cft_ent_entropy, label = 'CFT prediction')
plt.legend(loc = 'lower right', fontsize = 15)
plt.show()